# Sub Question Query Engine
In this tutorial, we showcase how to use a **sub question query engine** to tackle the problem of answering a complex query using multiple data sources.  
It first breaks down the complex query into sub questions for each relevant data source,
then gather all the intermediate reponses and synthesizes a final response.

### Preparation

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [2]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.0 MB/s eta 0:00:00


In [3]:
import os

os.environ["OPENAI_API_KEY"] = ""

import nest_asyncio

nest_asyncio.apply()

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

In [5]:
# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

In [24]:
!unzip /content/HowToPlayWeiss.zip

Archive:  /content/HowToPlayWeiss.zip
  inflating: HowToPlayWeiss/5HY_QuintsExampleItsuki_AI.txt  
  inflating: HowToPlayWeiss/BeginnersGuideToCreatingADeck.txt  
  inflating: HowToPlayWeiss/gameplay.txt  
  inflating: HowToPlayWeiss/GuideToPlayingAndPlayingAgainstMushokuTensei.txt  
  inflating: HowToPlayWeiss/HOL_HololiveGura_AI.txt  
  inflating: HowToPlayWeiss/Hololive Set 1 Cards HOL-xxx.txt  
  inflating: HowToPlayWeiss/MTI_MushokuTenseiExampleAI.txt  
   creating: HowToPlayWeiss/MTI_MushokuTenseiSet/
  inflating: HowToPlayWeiss/MTI_MushokuTenseiSet/MTI_S83-001_Magical_Talent_Rudeus.txt  
  inflating: HowToPlayWeiss/MTI_MushokuTenseiSet/MTI_S83-002_Live_Seriously_Rudeus.txt  
  inflating: HowToPlayWeiss/MTI_MushokuTenseiSet/MTI_S83-003_In_the_Sunlight_Rudeus.txt  
  inflating: HowToPlayWeiss/MTI_MushokuTenseiSet/MTI_S83-004_Adventuring_Party_Dead_End_Rudeus.txt  
  inflating: HowToPlayWeiss/MTI_MushokuTenseiSet/MTI_S83-005_Daily_Growth_Rudeus.txt  
  inflating: HowToPlayWeiss/MTI

In [25]:
# load data
humberviewbook = SimpleDirectoryReader(input_dir="./HowToPlayWeiss/").load_data()

# build index and query engine
vector_query_engine = VectorStoreIndex.from_documents(
    humberviewbook,
    use_async=True,
).as_query_engine()

**********
Trace: index_construction
    |_node_parsing -> 0.899646 seconds
      |_chunking -> 0.016252 seconds
      |_chunking -> 0.005318 seconds
      |_chunking -> 0.021566 seconds
      |_chunking -> 0.014214 seconds
      |_chunking -> 0.16931 seconds
      |_chunking -> 0.029051 seconds
      |_chunking -> 0.015897 seconds
      |_chunking -> 0.013396 seconds
      |_chunking -> 0.085253 seconds
      |_chunking -> 0.02457 seconds
      |_chunking -> 0.055943 seconds
      |_chunking -> 0.033603 seconds
      |_chunking -> 0.022067 seconds
      |_chunking -> 0.320901 seconds
      |_chunking -> 0.007108 seconds
    |_embedding -> 3.295832 seconds
    |_embedding -> 3.560258 seconds
    |_embedding -> 3.712632 seconds
**********


### Setup sub question query engine

In [63]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="WeissPlayer",
            description="Helps decide how to play weiss schawrz decks to win",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

In [64]:
promptHeader = "Header. Sections are \n ColorPriority: three colors out of (Red, Blue, Green, Yellow) ordered in order of level priority, where the first is what you want to level for level 1, then second for level 2, then third for level 3. Consider that to play a climax or level 1+ card you need the color in your level or clock, so if you have a cxcombo on a level 1 card level that color first and if you have a cxcombo on a level 3 card level that by level 2 or 3 "

### Run queries

In [103]:
def ask_weiss_schwarz_question(query_engine,decklist,series,query,starter):
    response = query_engine.query(
        f"""{starter}{query}
        """
    )
    print(response)

In [104]:
ask_weiss_schwarz_question(query_engine,decklist,series,"","which cards in the decklist are leve 3")

Generated 1 sub questions.
[WeissPlayer] Q: What are the level 3 cards in the decklist?
[WeissPlayer] A: The level 3 cards in the decklist are not explicitly mentioned in the provided context information.
**********
Trace: query
    |_query -> 5.224522 seconds
      |_llm -> 2.486996 seconds
      |_sub_question -> 1.71689 seconds
        |_query -> 1.716072 seconds
          |_retrieve -> 0.865906 seconds
            |_embedding -> 0.836344 seconds
          |_synthesize -> 0.84914 seconds
            |_templating -> 2.8e-05 seconds
            |_llm -> 0.839841 seconds
      |_synthesize -> 1.016987 seconds
        |_templating -> 2.7e-05 seconds
        |_llm -> 1.012992 seconds
**********
The level 3 cards in the decklist are not explicitly mentioned in the provided context information.


In [100]:
decklist = """
    MTI/S83-E101
MTI/S83-E025
MTI/S83-E025
MTI/S83-E025
MTI/S83-E054
MTI/S83-E054
MTI/S83-E054
MTI/S83-E054
MTI/S83-E083
MTI/S83-E083
MTI/S83-E083
MTI/S83-E083
MTI/S83-TE14
MTI/S83-TE14
MTI/S83-E081
MTI/S83-E081
MTI/S83-E081
MTI/S83-E081
MTI/S83-E055S
MTI/S83-E055S
MTI/S83-E052
MTI/S83-E052
MTI/S83-E052
MTI/S83-E052
MTI/S83-E084
MTI/S83-E084
MTI/S83-E098
MTI/S83-E098
MTI/S83-E098
MTI/S83-E057
MTI/S83-E057
MTI/S83-E056
MTI/S83-E010
MTI/S83-E058S
MTI/S83-E058S
MTI/S83-E058S
MTI/S83-E087S
MTI/S83-E087S
MTI/S83-E082
MTI/S83-E082
MTI/S83-E082
MTI/S83-E082
MTI/S83-E077
MTI/S83-E077
MTI/S83-E077
MTI/S83-E077
MTI/S83-E099R
MTI/S83-E099R
MTI/S83-E099
MTI/S83-E099
    """

series = "Mushoku Tensei(MTI)"
color = "Header. Sections are \n ColorPriority: three colors out of (Red, Blue, Green, Yellow) ordered in order of level priority, where the first is what you want to level for level 1, then second for level 2, then third for level 3. Consider that to play a climax or level 1+ card you need the color in your level or clock, so if you have a cxcombo on a level 1 card level that color first and if you have a cxcombo on a level 3 card level that by level 2 or 3 "
goingfirstatt = """GoingFirstAttacker section. as many lines as needed for matted as priority([number])[card name]. This the hardcoded priority levels for cards you can play on the first turn. Give cards that are
good to play on your first turn on the game a high priority number, like 500 and if a card should not be played then give it a negative value like -200. make sure all cards are level 0 and in the deck. Here is an example for set hololive

Section GoingFirst_Attacker
//good card to play at level 0
Priority(500) [Famous Detective, Watson Amelia]
Priority(-200) [A Step Towards the Future, AZKi] && Not Have(1,[A Step Towards the Future, AZKi])
//Don't play your only climax swapper turn 1 if you don't already have the right climax
Priority(-100) [Time Just for the Two, Pavolia Reine] && Not Have(1,[REPEAT THiS LiFE WiTH U]) && Not Have(1,[Time Just for the Two, Pavolia Reine])

Put at least 3 entries
"""
starter = f"""You are an expert weiss schwarz player. I will be asking you to script sections of code for a custom ai for a simulator, the format needed will be provided. When deciding on things
        consider what is the best choice to win the game, taking into consideration every card in the deck and their interactions. You will be coding a rule-based ai for one specific deck form series {series}, here is the deck: {decklist}.
        You have access to the color, level, soul, and effects of each card in the deck. DECISIONS ARE ONLY BASED ON CARDS IN THE DECK MAKE SURE THE CARDS ARE IN THE DECKLIST. consider them when making decisions. ask many questions, as many as needed. make a full confident answer that will best increase chances of winning.
        Please code the """